In [1]:
import transformers
from transformers import AutoTokenizer
import torch
import os
print(f"number of GPUs: torch.cuda.device_count()")
print(torch.__version__)

number of GPUs: torch.cuda.device_count()
2.6.0+rocm6.1


In [2]:
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct" 

if  torch.cuda.is_available():
    device = "cuda"
else:
    raise ValueError("No se reconoció GPU.")

pipeline = transformers.pipeline(
	"text-generation", 
	model=model_id,
	model_kwargs={"torch_dtype": torch.bfloat16},
	device=device
)

# Tokenizer necesario para contar tokens
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda


In [ ]:
# Directorios de entrada y salida
input_dir = "dev/"
output_dir = "out/"
os.makedirs(output_dir, exist_ok=True)

# Instrucciones para el modelo
prompt = [
    {"role": "system", 
     "content": 
        """ 
        Quiero que identifiques entidades nombradas que requieren ser anonimizadas en el informe clínico que copio entre comillas al final de esta instrucción. Quiero que me des el resultado en formato .xml in-line, donde las entidades sean identificadas por etiquetas en el mismo texto. Quiero que etiquetes con los criterios MEDDOCAN. A continuación, te muestro un ejemplo que contiene:
        - El texto original del informe en formato plano (.txt)
        - La representación estructurada del mismo en XML con etiquetas semánticas detalladas y posiciones de texto (atributos start, end, text, TYPE, etc.).
        Tu tarea será generar un XML con las mismas reglas de estructura y etiquetado a partir de cada texto clínico. Instrucciones:
        - Conserva el formato exacto del XML del ejemplo.
        - Cada etiqueta tiene que tener el tipo de entidad (`TYPE`) del inventario de MEDDOCAN. Los tipos de entidad que puedes usar son los siguientes: 
            NOMBRE_SUJETO_ASISTENCIA
            EDAD_SUJETO_ASISTENCIA
            SEXO_SUJETO_ASISTENCIA
            FAMILIARES_SUJETO_ASISTENCIA
            NOMBRE_PERSONAL_SANITARIO
            FECHAS
            PROFESION
            HOSPITAL
            CENTRO_SALUD
            INSTITUCION
            CALLE
            TERRITORIO
            PAIS
            NUMERO_TELEFONO
            NUMERO_FAX
            CORREO_ELECTRONICO
            ID_SUJETO_ASISTENCIA
            ID_CONTACTO_ASISTENCIAL
            ID_ASEGURAMIENTO
            ID_TITULACION_PERSONAL_SANITARIO
            ID_EMPLEO_PERSONAL_SANITARIO
            IDENTIF_VEHICULOS_NRSERIE_PLACAS
            IDENTIF_DISPOSITIVOS_NRSERIE
            DIREC_PROT_INTERNET
            URL_WEB
            IDENTIF_BIOMETRICOS
            OTRO_NUMERO_IDENTIF
            OTROS_SUJETO_ASISTENCIA
          - y un campo de comentario (`comment`) vacío
        Cuando te dé un nuevo texto, responde solo con el XML, sin explicaciones adicionales.
    
        Ejemplo - Informe en formato .txt: 
        Datos del paciente.
        Nombre: María Soledad Moreno Roca
        DNI: 23556552K
        Fecha de nacimiento: 09/01/1941
        Género: Mujer
        Domicilio: Calle de Almagro 80
        Ciudad: Denia, Valencia, Comunidad Valenciana
        Código postal: 46571
        Email: mariasoledad_roca@ucm.es
        Teléfono fijo: +34 960 66 89 48
        Teléfono móvil: +34 660 57 14 97
        NHC: 2409425
        NASS: 468043486571
        Condición de riesgo: Científico de Investigación
        
        Datos asistenciales.
        Médico: Dr. Juan Ramón Benito Vicente. NC 097900390. Investigador Clínico en Epidemiología. Instituto de Investigación Biomédica en Red de Enfermedades Infecciosas (CIBERINFEC). Avenida Monforte de Lemos 3-5. 28029. Madrid. España.
        Fecha de ingreso: 05/06/1996
        Centro de salud: Centro de Salud Carabanchel
        
        Informe clínico del paciente:
        Paciente sobreviviente de violencia de 55 años de edad, acompañado de su madre. 
        
        Ejemplo - Informe en formato .xml: lo que debes generar
        <?xml version='1.0' encoding='UTF-8'?>
        <MEDDOCAN>
          <TEXT>
        Ejemplo - Informe en formato .txt: 
        Datos del paciente.
        Nombre:  <TAG TYPE="NOMBRE_SUJETO_ASISTENCIA">María Soledad</TAG> <TAG TYPE="NOMBRE_SUJETO_ASISTENCIA">Moreno Roca</TAG>
        DNI: <TAG TYPE="ID_SUJETO_ASISTENCIA">23556552K</TAG>
        Fecha de nacimiento: <TAG TYPE="FECHAS">09/01/1941</TAG>
        Género: <TAG TYPE="SEXO_SUJETO_ASISTENCIA">Mujer</TAG>
        Domicilio: <TAG TYPE="CALLE">Calle de Almagro 80</TAG>.
        Ciudad: <TAG TYPE="TERRITORIO">Denia</TAG>, <TAG TYPE="TERRITORIO">Valencia</TAG>, <TAG TYPE="TERRITORIO">Comunidad Valenciana</TAG>
        Código postal: <TAG TYPE="TERRITORIO">46571</TAG>
        Email: <TAG TYPE="CORREO_ELECTRONICO">mariasoledad_roca@ucm.es</TAG>
        Teléfono fijo: <TAG TYPE="NUMERO_TELEFONO">+34 960 66 89 48</TAG>
        Teléfono móvil: <TAG TYPE="NUMERO_TELEFONO">+34 660 57 14 97</TAG>
        NHC: <TAG TYPE="ID_SUJETO_ASISTENCIA">2409425</TAG>
        NASS: <TAG TYPE="ID_ASEGURAMIENTO">468043486571</TAG>
        Condición de riesgo: <TAG TYPE="PROFESION">Científico de Investigación</TAG>
        
        Datos asistenciales.
        Médico: Dr. <TAG TYPE="NOMBRE_PERSONAL_SANITARIO">Juan Ramón Benito Vicente</TAG>. NC <TAG TYPE="ID_TITULACION_PERSONAL_SANITARIO">097900390</TAG>. <TAG TYPE="ID_EMPLEO_PERSONAL_SANITARIO">Investigador Clínico en Epidemiología</TAG>. <TAG TYPE="INSTITUCION">Instituto de Investigación Biomédica en Red de Enfermedades Infecciosas (CIBERINFEC)</TAG>. <TAG TYPE="CALLE">Avenida Monforte de Lemos 3-5</TAG>. <TAG TYPE="TERRITORIO">28029</TAG>. <TAG TYPE="TERRITORIO">Madrid</TAG>. <TAG TYPE="PAIS">España</TAG>.
        Fecha de ingreso: <TAG TYPE="FECHAS">05/06/1996</TAG>
        Centro de salud: <TAG TYPE="CENTRO_SALUD">Centro de Salud Carabanchel</TAG>
        
        Informe clínico del paciente:
        Paciente <TAG TYPE="OTROS_SUJETO_ASISTENCIA">sobreviviente de violencia</TAG> de <TAG TYPE="EDAD_SUJETO_ASISTENCIA">55 años</TAG> de edad, acompañado de su <TAG TYPE="FAMILIARES_SUJETO_ASISTENCIA">madre</TAG>.
            </TEXT>
        </MEDDOCAN>
    
        Recordá que en ningún caso debes incluir advertencias, explicaciones ni descripciones sobre la tarea, sobre la instrucción que te he dado o sobre cuestiones de funcionamiento del modelo de lenguaje.
        """},
     ]

# Configuración de tokens
MAX_CONTEXT_TOKENS = 8192
MAX_GENERATION_TOKENS = 4000
MAX_INPUT_TOKENS = MAX_CONTEXT_TOKENS - MAX_GENERATION_TOKENS



# Procesar cada archivo .txt
for filename in os.listdir(input_dir):
    if filename.endswith(".txt"):
        filepath = os.path.join(input_dir, filename)
        with open(filepath, "r", encoding="utf-8") as f:
            texto = f.read()

        # Crear mensaje estilo chat
        prompt_text = prompt[0]["content"]
        messages = [
            {"role": "system", "content": prompt_text},
            {"role": "user", "content": texto}
        ]

        # Calcular tokens de entrada
        full_prompt = prompt_text + texto
        total_tokens = len(tokenizer.encode(full_prompt))
        print(f"{filename}: Tokens de entrada: {total_tokens}")

        # Truncar el prompt si se pasa del límite permitido
        if total_tokens > MAX_INPUT_TOKENS:
            print(f"Truncando prompt: {filename}")
            # Calcular los tokens disponibles para el prompt
            max_tokens_prompt = MAX_INPUT_TOKENS - len(tokenizer.encode(texto))
            
            # Truncar el prompt para ajustarlo al límite de tokens
            prompt_tokens = tokenizer.encode(prompt[0]["content"])
            truncated_prompt_tokens = prompt_tokens[:max_tokens_prompt]
            
            # Decodificar los tokens truncados y actualizar el prompt
            truncated_prompt = tokenizer.decode(truncated_prompt_tokens, skip_special_tokens=True)
            messages[0]["content"] = truncated_prompt

        # Generar texto
        output = pipeline(messages, max_new_tokens=MAX_GENERATION_TOKENS)

        # Extraer solo el contenido generado por el modelo
        respuesta = output[0]["generated_text"][2]["content"]

        # Guardar en .xml
        output_filename = os.path.splitext(filename)[0] + ".xml"
        output_path = os.path.join(output_dir, output_filename)
        with open(output_path, "w", encoding="utf-8") as out_f:
            out_f.write(respuesta)

        print(f"Procesado: {filename} → {output_filename}")

print("Proceso completado.")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


721821248.txt: Tokens de entrada: 1964


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Procesado: 721821248.txt → 721821248.xml
640351088.txt: Tokens de entrada: 1993


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Procesado: 719529678.txt → 719529678.xml
551573814.txt: Tokens de entrada: 1948


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Procesado: 551573814.txt → 551573814.xml
723030912.txt: Tokens de entrada: 1960


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Procesado: 723030912.txt → 723030912.xml
605419284.txt: Tokens de entrada: 1985
Procesado: 605419284.txt → 605419284.xml
Proceso completado.
